In [1]:
"""
torchtext:#提供了文本处理的工具和数据集，常用于自然语言处理（NLP）任务
de_core_news_sm:#下载并安装spaCy的德语小型模型de_core_news_sm
en_core_web_sm:#下载并安装spaCy的英语小型模型
portalocker:#用于文件锁定的库，以防止多个进程同时写入同一个文件。
torchdata:#PyTorch的一个扩展库，提供了数据加载、预处理功能，以及多种数据集的接口
sacrebleu:#提供了标准化的BLEU分数计算方法，用于评估机器翻译和其他生成任务的性能。
torchsummaryX:#这个库为PyTorch模型提供了详细的结构摘要输出，包括每层的形状、参数数量等信息
"""
# # Uncomment to install
# !pip install -U torchtext -q 
# !python -m spacy download "de_core_news_sm" 
# !python -m spacy download "en_core_web_sm" 
# !pip install portalocker>=2.0 -q 
# !pip install -U torchdata -q 
# !pip install sacrebleu -q 
# !pip install torchsummaryX -q 
# # You may need to restart your runtime after this

'\ntorchtext:#提供了文本处理的工具和数据集，常用于自然语言处理（NLP）任务\nde_core_news_sm:#下载并安装spaCy的德语小型模型de_core_news_sm\nen_core_web_sm:#下载并安装spaCy的英语小型模型\nportalocker:#用于文件锁定的库，以防止多个进程同时写入同一个文件。\ntorchdata:#PyTorch的一个扩展库，提供了数据加载、预处理功能，以及多种数据集的接口\nsacrebleu:#提供了标准化的BLEU分数计算方法，用于评估机器翻译和其他生成任务的性能。\ntorchsummaryX:#这个库为PyTorch模型提供了详细的结构摘要输出，包括每层的形状、参数数量等信息\n'

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR

import sacrebleu
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Tuple
import torchdata
import spacy
import random

from torchsummaryX import summary

In [4]:
print(torch.__version__)

2.2.2+cu121


# 1 数据集和分词

In [5]:
# Use this cell if you get a UTF Encoding Error
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [12]:
# 创建文件夹，并将数据集文件夹中相关文件解压并放到刚创建的文件中

import os
import gzip
import shutil

# 创建所需的文件夹
folders = ['Multi30k', 'Multi30k/train', 'Multi30k/val', 'Multi30k/test']
for folder in folders:
    os.makedirs(folder, exist_ok=True)

# 定义一个解压缩并复制文件的函数
def gunzip_shutil(source_filepath, dest_filepath):
    with gzip.open(source_filepath, 'rb') as f_in:
        with open(dest_filepath, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

# 执行解压缩
source_base_path = 'E:/project_2023/multi30k-dataset/data/task1/raw'
dest_base_path = 'Multi30k'
file_pairs = [
    ('train.en.gz', 'train/train.en'),
    ('train.de.gz', 'train/train.de'),
    ('val.en.gz', 'val/val.en'),
    ('val.de.gz', 'val/val.de'),
    ('test_2016_flickr.en.gz', 'test/test.en'),
    ('test_2016_flickr.de.gz', 'test/test.de')
]

for source_file, dest_file in file_pairs:
    source_filepath = os.path.join(source_base_path, source_file)
    dest_filepath = os.path.join(dest_base_path, dest_file)
    gunzip_shutil(source_filepath, dest_filepath)



In [1]:
# 导入必要的库
from torchtext.data.utils import get_tokenizer  # 用于获取分词器
from torchtext.datasets import Multi30k  # Multi30k数据集
from collections import Counter  # 计数器，用于统计词频
from tqdm import tqdm  # 进度条库，用于显示进度

root = '/content/Multi30k/'  # 定义数据集的根目录

# 获取英文和德文的分词器，使用的是spacy库的分词器
en_tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
de_tokenizer = get_tokenizer("spacy", language="de_core_news_sm")

# 定义英文文本的分词函数
def tokenize_en(text):
    doc = en_tokenizer(str(text))
    return [token for token in doc]

# 定义德文文本的分词函数
def tokenize_de(text):
    doc = de_tokenizer(str(text))
    return [token for token in doc]

# 定义词汇表类
class Vocab:

    def __init__(self, tokenizer, min_freq=2, data=None, special_tokens=['<pad>', '<sos>', '<eos>', '<unk>']):
        # 初始化词汇表对象
        self.tokenizer = tokenizer  # 分词器
        self.min_freq = min_freq  # 最小词频阈值
        self.special_tokens = special_tokens  # 特殊标记列表
        self.build_vocab(data)  # 构建词汇表

    def build_vocab(self, data):
        # 构建词汇表的方法
        counter = Counter()  # 创建计数器
        for text in tqdm(data):
            tokens = self.tokenizer(text)  # 对文本进行分词
            counter.update(tokens)  # 更新词频统计

        # 过滤掉词频小于最小词频阈值的词
        tokens = [token for token, freq in counter.items() if freq >= self.min_freq]

        # 在词列表前加上特殊标记
        tokens = self.special_tokens + tokens

        # 创建字符串到索引的映射字典
        self.stoi = {token: index for index, token in enumerate(tokens)}
        # 索引到字符串的映射列表
        self.itos = tokens  

    def __len__(self):
        # 返回词汇表的大小
        return len(self.stoi)

    def __getitem__(self, token):
        # 根据词获取其索引，如果不存在则返回'<unk>'的索引
        return self.stoi.get(token, self.stoi['<unk>'])

en_file = "Multi30k/train/train.en"  # 英文文件路径
de_file = "Multi30k/train/train.de"  # 德文文件路径

# 打开英文文件并读取内容
with open(en_file, "r", encoding="utf8") as f:
    train_data_en = [text.strip() for text in f.readlines()]

# 打开德文文件并读取内容
with open(de_file, "r", encoding="utf8") as f:
    train_data_de = [text.strip() for text in f.readlines()]

# 使用英文和德文的训练数据创建词汇表对象
EN_VOCAB = Vocab(tokenize_en, min_freq=1, data=train_data_en)
DE_VOCAB = Vocab(tokenize_de, min_freq=1, data=train_data_de)

# 打印英文和德文词汇表的大小
print("\nVocab Size English", len(EN_VOCAB))
print("\nVocab Size German", len(DE_VOCAB))


100%|██████████| 29000/29000 [00:00<00:00, 30010.73it/s]


Vocab Size English 10837

Vocab Size German 19214


In [4]:
EN_VOCAB

定义一个自定义的PyTorch数据集TranslationDataset，用于加载和准备机器翻译任务的数据。

In [8]:
# 导入PyTorch相关库
import torch
from torch.utils.data import Dataset, DataLoader

# 定义一个用于机器翻译的数据集类，继承自PyTorch的Dataset类
class TranslationDataset(Dataset):

    # 构造函数初始化数据集对象
    def __init__(self, en_data, de_data, src_tokenizer, tgt_tokenizer, src_vocab, tgt_vocab):
        # 初始化英文数据列表
        self.en_data = en_data
        # 初始化德文数据列表
        self.de_data = de_data
        # 源语言（英文）的分词器
        self.src_tokenizer = src_tokenizer
        # 目标语言（德文）的分词器
        self.tgt_tokenizer = tgt_tokenizer
        # 源语言的词汇表，用于将单词转换为索引
        self.src_vocab = src_vocab
        # 目标语言的词汇表，用于将单词转换为索引
        self.tgt_vocab = tgt_vocab

    # 该方法根据索引获取数据集中的特定项目
    def __getitem__(self, index):
        # 根据索引获取源语言和目标语言的文本
        src_txt, tgt_txt = self.en_data[index], self.de_data[index]

        # 使用分词器对文本进行分词，并使用词汇表将单词转换为索引
        src_tokens = [self.src_vocab[token] for token in self.src_tokenizer(src_txt)]
        tgt_tokens = [self.tgt_vocab[token] for token in self.tgt_tokenizer(tgt_txt)]

        # 在序列的开始和结束分别添加特殊标记<sos>和<eos>
        src_tokens = [self.src_vocab['<sos>']] + src_tokens + [self.src_vocab['<eos>']]
        tgt_tokens = [self.tgt_vocab['<sos>']] + tgt_tokens + [self.tgt_vocab['<eos>']]

        # 将索引列表转换为PyTorch张量
        src_tensor = torch.LongTensor(src_tokens)
        tgt_tensor = torch.LongTensor(tgt_tokens)

        # 返回处理好的源语言和目标语言张量
        return src_tensor, tgt_tensor

    # 该方法返回数据集中的项目总数
    def __len__(self):
        # 确保源语言和目标语言数据列表的长度相等
        assert len(self.en_data) == len(self.de_data)
        # 返回数据集的大小
        return len(self.en_data)

    # 自定义批处理函数，用于数据加载器在加载批次数据时的数据处理
    def collate_fn(self, batch):
        # 将批次中的数据分解为源语言和目标语言张量列表
        src_tensors, tgt_tensors = zip(*batch)
        # 使用pad_sequence对序列进行填充，使得批次中的所有序列长度相等，填充值为<pad>标记的索引
        src_tensors = torch.nn.utils.rnn.pad_sequence(src_tensors, padding_value=self.src_vocab['<pad>'], batch_first=True)
        tgt_tensors = torch.nn.utils.rnn.pad_sequence(tgt_tensors, padding_value=self.tgt_vocab['<pad>'], batch_first=True)

        # 返回处理好的源语言和目标语言批次数据
        return src_tensors, tgt_tensors


In [9]:
# 定义训练集文件路径
en_file = "Multi30k/train/train.en"
de_file = "Multi30k/train/train.de"

# 打开英文训练集文件，并读取其内容。去除每行文本的首尾空白字符。
with open(en_file, "r", encoding="utf8") as f:
    train_data_en = [text.strip() for text in f.readlines()]

# 打开德文训练集文件，并读取其内容。同样去除每行文本的首尾空白字符。
with open(de_file, "r", encoding="utf8") as f:
    train_data_de = [text.strip() for text in f.readlines()]



# 定义验证集文件路径
en_file = "Multi30k/val/val.en"
de_file = "Multi30k/val/val.de"

# 打开英文验证集文件，并读取其内容，处理方式同上。
with open(en_file, "r", encoding="utf8") as f:
    val_data_en = [text.strip() for text in f.readlines()]

# 打开德文验证集文件，并读取其内容，处理方式同上。
with open(de_file, "r", encoding="utf8") as f:
    val_data_de = [text.strip() for text in f.readlines()]



# 定义测试集文件路径
en_file = "Multi30k/test/test.en"
de_file = "Multi30k/test/test.de"

# 打开英文测试集文件，并读取其内容，处理方式同上。
with open(en_file, "r", encoding="utf8") as f:
    test_data_en = [text.strip() for text in f.readlines()]

# 打开德文测试集文件，并读取其内容，处理方式同上。
with open(de_file, "r", encoding="utf8") as f:
    test_data_de = [text.strip() for text in f.readlines()]




# 使用读取的训练集数据创建TranslationDataset对象。
train_dataset = TranslationDataset(train_data_en, train_data_de, tokenize_en, tokenize_de, EN_VOCAB, DE_VOCAB)

# 使用读取的验证集数据创建TranslationDataset对象。
val_dataset = TranslationDataset(val_data_en, val_data_de, tokenize_en, tokenize_de, EN_VOCAB, DE_VOCAB)

# 使用读取的测试集数据创建TranslationDataset对象。
test_dataset = TranslationDataset(test_data_en, test_data_de, tokenize_en, tokenize_de, EN_VOCAB, DE_VOCAB)

# 定义批处理大小。
BATCH_SIZE = 128

# 创建DataLoader对象，用于训练集数据的批量加载和打乱。
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=train_dataset.collate_fn)

# 创建DataLoader对象，用于验证集数据的批量加载，不打乱数据。
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=val_dataset.collate_fn)

# 创建DataLoader对象，用于测试集数据的批量加载，不打乱数据。
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=test_dataset.collate_fn)


### 查看序列实例

In [11]:
len(train_dataset)

29000

In [8]:
train_dataset[0]

(tensor([ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  2]),
 tensor([ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  2]))

In [9]:
' '.join([EN_VOCAB.itos[i] for i in train_dataset[0][0]]), ' '.join([DE_VOCAB.itos[i] for i in train_dataset[0][1]])

('<sos> Two young , White males are outside near many bushes . <eos>',
 '<sos> Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche . <eos>')

In [10]:
test_data_en[0], test_data_de[0]

('A man in an orange hat starring at something.',
 'Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.')

In [11]:
' '.join([EN_VOCAB.itos[i] for i in test_dataset[0][0]]), ' '.join([DE_VOCAB.itos[i] for i in test_dataset[0][1]])

('<sos> A man in an orange hat starring at something . <eos>',
 '<sos> Ein Mann mit einem orangefarbenen Hut , der etwas <unk> . <eos>')

# 2 模型结构

## 2.1 位置编码 

In [23]:
# 导入数学库
import math

# PositionalEncoding类继承自nn.Module，是一个PyTorch模块
class PositionalEncoding(nn.Module):

    # 类的初始化方法
    def __init__(self, d_model, max_seq_len, dropout=0.1):
        """
        初始化PositionalEncoding模块。

        参数:
        d_model (int): 输入向量的维度。
        max_seq_len (int): 输入序列的最大长度。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        # 调用父类的初始化方法
        super(PositionalEncoding, self).__init__()
        # 定义一个dropout层，用于在位置编码加到输入向量之后进行dropout操作
        self.dropout = nn.Dropout(dropout)

        # 初始化一个位置编码矩阵，全为零，形状为(max_seq_len, d_model)
        pe = torch.zeros(max_seq_len, d_model)

        # 生成一个位置索引向量，形状为(max_seq_len, 1)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)

        # 生成除数项，用于计算正弦和余弦函数的参数，形状为(d_model / 2,)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))

        # 计算位置编码的正弦部分，填充到偶数位置
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算位置编码的余弦部分，填充到奇数位置
        pe[:, 1::2] = torch.cos(position * div_term)
        # 增加一个维度，使pe的形状为(1, max_seq_len, d_model)，方便后续的广播操作
        pe = pe.unsqueeze(0)

        # 将位置编码矩阵注册为一个buffer，这样它就可以被保存在模型的state_dict中了，
        # 虽然它不是一个可学习的参数。
        self.register_buffer('pe', pe)

    # 定义前向传播方法
    def forward(self, x):
        """
        PositionalEncoding的前向传播方法。

        参数:
        x (Tensor): 输入张量，形状为(batch_size, seq_length, d_model)。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        # 将位置编码加到输入张量上，使用广播机制匹配序列长度
        x = x + self.pe[:, :x.size(1), :]
        # 对加上位置编码后的输入进行dropout操作
        x = self.dropout(x)
        # 返回处理后的张量
        return x


## 2.2 Multi-Head Attention模块

In [24]:
# 导入PyTorch库
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义多头自注意力模块，继承自nn.Module
class MultiHeadSelfAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        """
        初始化多头自注意力模块。

        参数:
        d_model (int): 输入张量的维度。
        num_heads (int): 注意力头的数量。
        """
        super(MultiHeadSelfAttention, self).__init__()
        self.d_model = d_model  # 保存输入维度
        self.num_heads = num_heads  # 保存头的数量
        self.head_dim = d_model // num_heads  # 计算每个头的维度

        # 确保d_model能被num_heads整除，保证等分
        assert self.head_dim * num_heads == d_model, "d_model must be divisible by num_heads"

        # 定义全连接层，用于生成查询、键、值
        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)

        # 定义输出全连接层
        self.wo = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        """
        多头自注意力的前向传播。

        参数:
        q (Tensor): 查询张量，形状为(batch_size, seq_length, d_model)。
        k (Tensor): 键张量，形状为(batch_size, seq_length, d_model)。
        v (Tensor): 值张量，形状为(batch_size, seq_length, d_model)。
        mask (Tensor, optional): 掩码张量，用于忽略某些元素，默认为None。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        batch_size = q.size(0)

        # 将查询、键、值通过全连接层后，改变形状以适应多头处理
        q = self.wq(q).view(batch_size, -1, self.num_heads, self.head_dim)
        k = self.wk(k).view(batch_size, -1, self.num_heads, self.head_dim)
        v = self.wv(v).view(batch_size, -1, self.num_heads, self.head_dim)

        # 转置操作，以满足矩阵乘法的需求
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # 计算注意力权重矩阵
        attn = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)

        # 如果提供了掩码，则应用掩码
        if mask is not None:
            attn = attn.masked_fill(mask == 0, float('-inf'))

        # 应用softmax获取标准化的注意力权重
        attn = F.softmax(attn, dim=-1)

        # 根据注意力权重计算输出值
        out = torch.matmul(attn, v).transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        # 将输出通过最后一个全连接层
        out = self.wo(out)

        return out


## 2.3 Add 和 Norm 模块

In [25]:
# AddNorm类继承自nn.Module，是PyTorch中的标准写法
class AddNorm(nn.Module):

    def __init__(self, d_model, eps=1e-6):
        """
        初始化AddNorm模块。

        参数:
        d_model (int): 输入张量的维度。
        eps (float, optional): 为了数值稳定性而加入的一个小常数，默认值为1e-6。
        """
        super(AddNorm, self).__init__()  # 调用父类的初始化函数
        # 初始化一个层归一化（LayerNorm）模块，它会对输入张量的最后一维进行归一化
        self.norm = nn.LayerNorm(d_model, eps=eps)

    def forward(self, x, residual):
        """
        AddNorm的前向传播函数。

        参数:
        x (Tensor): 输入张量，形状为(batch_size, seq_length, d_model)。
        residual (Tensor): 残差张量，与输入张量形状相同。

        返回:
        Tensor: 输出张量，形状也为(batch_size, seq_length, d_model)。
        """
        # 首先进行残差连接，即将输入张量和残差张量相加
        out = x + residual
        # 然后对相加后的结果进行层归一化处理
        out = self.norm(out)

        # 返回归一化后的输出张量
        return out


## 2.4 Feed-Forward 模块

In [26]:
# PositionwiseFeedForward类继承自nn.Module，是一个PyTorch模块
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        初始化逐位置前馈网络模块。

        参数:
        d_model (int): 输入张量的维度。
        d_ff (int): 前馈网络隐藏层的维度。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        super(PositionwiseFeedForward, self).__init__()
        # 定义第一个全连接层，将输入维度从d_model映射到d_ff
        self.linear1 = nn.Linear(d_model, d_ff)
        # 定义一个dropout层，用于防止过拟合
        self.dropout = nn.Dropout(dropout)
        # 定义第二个全连接层，将维度从d_ff映射回d_model
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        """
        逐位置前馈网络的前向传播函数。

        参数:
        x (Tensor): 输入张量，形状为(batch_size, seq_length, d_model)。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        # 通过第一个全连接层
        out = self.linear1(x)
        # 应用ReLU激活函数
        out = F.relu(out)
        # 应用dropout
        out = self.dropout(out)
        # 通过第二个全连接层
        out = self.linear2(out)

        # 返回最终的输出张量
        return out


## 2.5 Encoder 类
- Multi-Head Self-Attention layer
- Add & Norm (Residual connection and Layer Normalization)
- Position-wise Feed-Forward Network layer
- Add & Norm (Residual connection and Layer Normalization)

In [27]:
# EncoderBlock类继承自nn.Module，是一个PyTorch模块
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        初始化编码器块模块。

        参数:
        d_model (int): 输入张量的维度。
        num_heads (int): 注意力机制中头的数量。
        d_ff (int): 前馈网络中隐藏层的维度。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        super(EncoderBlock, self).__init__()
        # 初始化多头自注意力层
        self.self_attn = MultiHeadSelfAttention(d_model, num_heads)
        # 初始化第一个添加残差连接和层归一化的模块
        self.norm1 = AddNorm(d_model)
        # 初始化逐位置前馈网络
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        # 初始化第二个添加残差连接和层归一化的模块
        self.norm2 = AddNorm(d_model)

    def forward(self, x, mask=None):
        """
        编码器块的前向传播函数。

        参数:
        x (Tensor): 输入张量，形状为(batch_size, seq_length, d_model)。
        mask (Tensor, optional): 掩码张量，用于忽略某些元素，默认为None。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        # 通过多头自注意力层处理输入
        x1 = self.self_attn(q=x, k=x, v=x, mask=mask)
        # 应用残差连接和层归一化
        x = self.norm1(x, x1)
        # 通过逐位置前馈网络处理
        x1 = self.ffn(x)
        # 再次应用残差连接和层归一化
        x = self.norm2(x, x1)

        # 返回最终的输出
        return x


## 2.6 Decoder 类
- Masked Multi-Head Self-Attention layer
- Add & Norm (Residual connection and Layer Normalization)
- Encoder-Decoder Multi-Head Attention layer
- Add & Norm (Residual connection and Layer Normalization)
- Position-wise Feed-Forward Network layer
- Add & Norm (Residual connection and Layer Normalization)

In [28]:
# DecoderBlock类继承自nn.Module，是一个PyTorch模块
class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        初始化解码器块模块。

        参数:
        d_model (int): 输入张量的维度。
        num_heads (int): 注意力机制中头的数量。
        d_ff (int): 前馈网络中隐藏层的维度。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        super(DecoderBlock, self).__init__()
        # 初始化自注意力层
        self.self_attn = MultiHeadSelfAttention(d_model, num_heads)
        # 初始化第一个残差连接和层归一化
        self.norm1 = AddNorm(d_model)
        # 初始化编码器-解码器注意力层，用于关注编码器的输出
        self.enc_dec_attn = MultiHeadSelfAttention(d_model, num_heads)
        # 初始化第二个残差连接和层归一化
        self.norm2 = AddNorm(d_model)
        # 初始化逐位置前馈网络
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        # 初始化第三个残差连接和层归一化
        self.norm3 = AddNorm(d_model)

    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        """
        解码器块的前向传播函数。

        参数:
        x (Tensor): 目标输入张量，形状为(batch_size, seq_length, d_model)。
        enc_output (Tensor): 编码器输出张量，形状为(batch_size, seq_length, d_model)。
        src_mask (Tensor, optional): 来源掩码张量，用于忽略某些元素，针对编码器输出，默认为None。
        tgt_mask (Tensor, optional): 目标掩码张量，用于忽略某些元素，针对自注意力层，默认为None。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        # 通过自注意力层处理输入
        x1 = self.self_attn(q=x, k=x, v=x, mask=tgt_mask)
        # 应用残差连接和层归一化
        x = self.norm1(x, x1)
        # 通过编码器-解码器注意力层处理，关注编码器的输出
        x1 = self.enc_dec_attn(q=x, k=enc_output, v=enc_output, mask=src_mask)
        # 再次应用残差连接和层归一化
        x = self.norm2(x, x1)
        # 通过逐位置前馈网络处理
        x1 = self.ffn(x)
        # 最后再次应用残差连接和层归一化
        x = self.norm3(x, x1)

        # 返回最终的输出
        return x


## 2.7 transformer 类

In [29]:
# Transformer类继承自nn.Module，是一个PyTorch模块
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, D_MODEL, num_heads, d_ff, max_seq_len, num_layers, dropout=0.1):
        """
        初始化Transformer模块。

        参数:
        src_vocab_size (int): 源词汇表的大小。
        tgt_vocab_size (int): 目标词汇表的大小。
        D_MODEL (int): 嵌入向量的维度。
        num_heads (int): 注意力机制中头的数量。
        d_ff (int): 前馈网络中隐藏层的维度。
        max_seq_len (int): 输入序列的最大长度。
        num_layers (int): 编码器和解码器中层的数量。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        super(Transformer, self).__init__()
        # 初始化源词嵌入层
        self.src_embedding = nn.Embedding(src_vocab_size, D_MODEL)
        # 初始化目标词嵌入层
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, D_MODEL)
        # 初始化位置编码
        self.pos_encoding = PositionalEncoding(D_MODEL, max_seq_len, dropout)

        # 初始化编码器层，使用ModuleList容纳多个EncoderBlock实例
        self.encoder_layers = nn.ModuleList([EncoderBlock(D_MODEL, num_heads, d_ff, dropout) for _ in range(num_layers)])
        # 初始化解码器层，同样使用ModuleList
        self.decoder_layers = nn.ModuleList([DecoderBlock(D_MODEL, num_heads, d_ff, dropout) for _ in range(num_layers)])

        # 初始化一个全连接层，用于将解码器的输出映射到目标词汇空间
        self.fc = nn.Linear(D_MODEL, tgt_vocab_size)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        """
        Transformer的前向传播函数。

        参数:
        src (Tensor): 源输入张量，形状为(batch_size, src_seq_length)。
        tgt (Tensor): 目标输入张量，形状为(batch_size, tgt_seq_length)。
        src_mask (Tensor, optional): 源掩码张量，用于忽略某些元素，默认为None。
        tgt_mask (Tensor, optional): 目标掩码张量，用于忽略某些元素，默认为None。

        返回:
        Tensor: 输出张量，形状为(batch_size, tgt_seq_length, tgt_vocab_size)。
        """
        # 对源序列进行词嵌入和位置编码
        src = self.src_embedding(src)
        src = self.pos_encoding(src)

        # 对目标序列进行词嵌入和位置编码
        tgt = self.tgt_embedding(tgt)
        tgt = self.pos_encoding(tgt)

        # 依次通过编码器层
        for layer in self.encoder_layers:
            src = layer(src, src_mask)

        # 依次通过解码器层，需要提供编码器的输出和掩码信息
        for layer in self.decoder_layers:
            tgt = layer(tgt, src, src_mask, tgt_mask)

        # 通过最后的全连接层，映射到目标词汇空间
        out = self.fc(tgt)

        return out


# 训练设置

## 超参数和模型

In [30]:
# 定义训练时的超参数
NUM_EPOCHS      = 20            # 训练的轮数
D_MODEL         = 256           # 模型中嵌入向量的维度
ATTN_HEADS      = 8             # 多头注意力中头的数量
NUM_LAYERS      = 3             # 编码器和解码器层重复的次数
FEEDFORWARD_DIM = 512           # 前馈网络中隐藏层的维度
DROPOUT         = 0.1           # Dropout概率，用于防止过拟合
MAX_SEQ_LEN     = 150           # 输入序列的最大长度
SRC_VOCAB_SIZE  = len(EN_VOCAB)  # 源语言词汇表的大小
TGT_VOCAB_SIZE  = len(DE_VOCAB)  # 目标语言词汇表的大小
LR              = 0             # 学习率，这里设置为0可能是为了后续调整

# 设备配置，优先使用GPU，如果没有GPU则使用CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [31]:
# NoamScheduler类负责根据训练步骤动态调整学习率
class NoamScheduler:

    def __init__(self, optimizer, d_model, warmup_steps=4000):
        """
        初始化NoamScheduler。

        参数:
        optimizer: 用于训练的优化器。
        d_model (int): 模型中嵌入向量的维度。
        warmup_steps (int): 预热步数，在此步数之前学习率会线性增加。
        """
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.current_step = 0  # 初始化当前步数

    def step(self):
        """
        更新学习率并增加步数。
        """
        self.current_step += 1  # 每调用一次，步数增加1
        lr = self.learning_rate()  # 计算当前步数的学习率
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr  # 更新优化器中的学习率

    def learning_rate(self):
        """
        根据当前步数计算学习率。
        """
        step = self.current_step
        # 学习率随着步数先增后减，增加部分线性增加至warmup_steps，之后随步数的增加而减小
        return (self.d_model ** -0.5) * min(step ** -0.5, step * self.warmup_steps ** -1.5)


In [32]:
# 初始化Transformer模型，并将其移动到适当的设备上
model = Transformer(SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, D_MODEL, ATTN_HEADS, FEEDFORWARD_DIM, MAX_SEQ_LEN, NUM_LAYERS, DROPOUT).to(DEVICE)

# 初始化优化器，这里使用AdamW优化器，它是Adam优化器的一个变种，添加了权重衰减
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9, weight_decay=5e-2)

# 根据训练数据加载器的长度计算预热步数
warmup_steps = 2 * len(train_dataloader)

# 使用NoamScheduler作为学习率调度器
scheduler = NoamScheduler(optimizer, d_model=D_MODEL, warmup_steps=warmup_steps)

# 初始化损失函数，使用交叉熵损失，并忽略'<pad>'标记的损失，同时应用标签平滑
criterion = torch.nn.CrossEntropyLoss(ignore_index=DE_VOCAB['<pad>'], label_smoothing=0.1)

# 初始化梯度缩放器，用于混合精度训练，有助于提高训练速度和减少内存消耗
scaler = torch.cuda.amp.GradScaler()


## 辅助函数

In [33]:
import sacrebleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

def generate_tgt_mask(tgt, pad_idx):
    """
    生成目标序列的掩码，遮蔽未来位置以及填充位置。
    
    参数:
    tgt (Tensor): 目标序列。
    pad_idx (int): `<pad>`标记的索引。
    
    返回:
    Tensor: 组合掩码。
    """
    seq_len = tgt.size(1)
    # 生成一个下三角矩阵，用于屏蔽未来的信息
    no_future_mask = torch.tril(torch.ones((seq_len, seq_len), device=DEVICE)).bool()
    # 生成一个填充位置的掩码
    pad_mask = (tgt != pad_idx).unsqueeze(1).unsqueeze(2)
    # 将两个掩码相与，得到最终的掩码
    combined_mask = pad_mask & no_future_mask
    return combined_mask


def generate_src_mask(src, pad_idx):
    """
    生成源序列的掩码，遮蔽填充位置。
    
    参数:
    src (Tensor): 源序列。
    pad_idx (int): `<pad>`标记的索引。
    
    返回:
    Tensor: 源序列的掩码。
    """
    mask = (src != pad_idx).unsqueeze(1).unsqueeze(2)
    return mask


def calculate_bleu(tgt_output, output):
    """
    计算BLEU分数。
    
    参数:
    tgt_output (Tensor): 真实的目标序列。
    output (Tensor): 模型生成的输出序列。
    
    返回:
    float: BLEU分数。
    """
    tgt_output = tgt_output.cpu().numpy()
    output = output.cpu().numpy()

    refs = []  # 参考序列
    hyps = []  # 假设序列

    # 需要排除的标记
    excluded_tokens = (DE_VOCAB['<pad>'], DE_VOCAB['<eos>'], DE_VOCAB['<sos>'])
    for tgt, pred in zip(tgt_output, output):
        # 将索引转换为单词，排除特定标记
        ref = ' '.join([DE_VOCAB.itos[t] for t in tgt if t not in excluded_tokens])
        hyp = ' '.join([DE_VOCAB.itos[t] for t in pred if t not in excluded_tokens])

        refs.append(ref)
        hyps.append(hyp)

    # 计算BLEU分数
    bleu = sacrebleu.corpus_bleu(hyps, [refs], force=True).score
    return bleu


## 模型结构

In [34]:
src, tgt = next(iter(train_dataloader))
src, tgt = src.to(DEVICE), tgt.to(DEVICE)

src_mask = generate_src_mask(src, EN_VOCAB['<pad>'])

tgt_input = tgt[:, :-1]
tgt_output = tgt[:, 1:]
tgt_mask = generate_tgt_mask(tgt_input, DE_VOCAB['<pad>'])

summary(model, src, tgt_input, src_mask, tgt_mask)

                                            Kernel Shape      Output Shape  \
Layer                                                                        
0_src_embedding                             [256, 10837]    [128, 25, 256]   
1_pos_encoding.Dropout_dropout                         -    [128, 25, 256]   
2_tgt_embedding                             [256, 19214]    [128, 27, 256]   
3_pos_encoding.Dropout_dropout                         -    [128, 27, 256]   
4_encoder_layers.0.self_attn.Linear_wq        [256, 256]    [128, 25, 256]   
5_encoder_layers.0.self_attn.Linear_wk        [256, 256]    [128, 25, 256]   
6_encoder_layers.0.self_attn.Linear_wv        [256, 256]    [128, 25, 256]   
7_encoder_layers.0.self_attn.Linear_wo        [256, 256]    [128, 25, 256]   
8_encoder_layers.0.norm1.LayerNorm_norm            [256]    [128, 25, 256]   
9_encoder_layers.0.ffn.Linear_linear1         [256, 512]    [128, 25, 512]   
10_encoder_layers.0.ffn.Dropout_dropout                -    [128

e:\soft_big\anaconda3\envs\torch12\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_src_embedding,"[256, 10837]","[128, 25, 256]",2774272.0,2774272.0
1_pos_encoding.Dropout_dropout,-,"[128, 25, 256]",NaN,NaN
2_tgt_embedding,"[256, 19214]","[128, 27, 256]",4918784.0,4918784.0
3_pos_encoding.Dropout_dropout,-,"[128, 27, 256]",NaN,NaN
4_encoder_layers.0.self_attn.Linear_wq,"[256, 256]","[128, 25, 256]",65792.0,65536.0
...,...,...,...,...
69_decoder_layers.2.ffn.Linear_linear1,"[256, 512]","[128, 27, 512]",131584.0,131072.0
70_decoder_layers.2.ffn.Dropout_dropout,-,"[128, 27, 512]",NaN,NaN
71_decoder_layers.2.ffn.Linear_linear2,"[512, 256]","[128, 27, 256]",131328.0,131072.0


## 训练和验证函数

In [35]:
def train_epoch(model, dataloader, optimizer, criterion, device):
    """
    对模型进行一个epoch的训练。

    参数:
    model: 训练的模型。
    dataloader: 数据加载器，提供训练数据。
    optimizer: 优化器，用于更新模型参数。
    criterion: 损失函数。
    device: 训练使用的设备（CPU或GPU）。

    返回:
    float: 该epoch的平均损失。
    """
    model.train()  # 将模型设置为训练模式
    total_loss = 0  # 记录总损失

    # 使用tqdm库显示训练进度条
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True,
                     leave=False, position=0, desc='Train')

    for i, (src, tgt) in enumerate(dataloader):
        # 将数据移动到指定的设备上
        src, tgt = src.to(device), tgt.to(device)
        # 生成源序列和目标序列的掩码
        src_mask = generate_src_mask(src, EN_VOCAB['<pad>'])
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]
        tgt_mask = generate_tgt_mask(tgt_input, DE_VOCAB['<pad>'])

        optimizer.zero_grad()  # 清空梯度

        # 使用混合精度训练
        with torch.cuda.amp.autocast():
            output = model(src, tgt_input, src_mask, tgt_mask)
            loss = criterion(output.reshape(-1, output.size(2)), tgt_output.reshape(-1))

        # 反向传播和优化
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        scheduler.step()  # 更新学习率

        total_loss += loss.item()
        # 更新进度条
        batch_bar.set_postfix(
            loss="{:.04f}".format(total_loss / (i + 1)),
            lr="{:.09f}".format(float(optimizer.param_groups[0]['lr'])))
        batch_bar.update()

    return total_loss / len(dataloader)


def validate_epoch(model, dataloader, criterion, DEVICE):
    """
    对模型进行一个epoch的验证。

    参数:
    model: 需要验证的模型。
    dataloader: 数据加载器，提供验证数据。
    criterion: 损失函数。
    DEVICE: 验证使用的设备（CPU或GPU）。

    返回:
    tuple: 包含平均损失和平均BLEU分数的元组。
    """
    model.eval()  # 将模型设置为评估模式
    epoch_loss = 0
    epoch_bleu_score = 0

    # 使用tqdm库显示验证进度条
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True,
                     leave=False, position=0, desc='Validate')

    with torch.no_grad():  # 在验证过程中不计算梯度
        for i, (src, tgt) in enumerate(dataloader):
            src, tgt = src.to(DEVICE), tgt.to(DEVICE)
            src_mask = generate_src_mask(src, EN_VOCAB['<pad>'])
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]
            tgt_mask = generate_tgt_mask(tgt_input, DE_VOCAB['<pad>'])

            # 使用混合精度评估
            with torch.cuda.amp.autocast():
                output = model(src, tgt_input, src_mask, tgt_mask)
                loss = criterion(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))

            epoch_loss += loss.item()
            # 计算并累加BLEU分数
            epoch_bleu_score += calculate_bleu(tgt_output, output.argmax(-1))

            # 更新进度条
            batch_bar.set_postfix(
                loss="{:.04f}".format(epoch_loss / (i + 1)),
                bleu="{:.04f}".format(epoch_bleu_score / (i + 1)))
            batch_bar.update()

    # 计算平均损失和BLEU分数
    epoch_loss /= len(dataloader)
    epoch_bleu_score /= len(dataloader)

    return epoch_loss, epoch_bleu_score


## 推理函数

In [36]:
def inference(model, src, de_tokenizer):
    """
    使用训练好的模型进行推理，生成目标语言序列。

    参数:
    model: 训练好的Transformer模型。
    src (Tensor): 源语言序列的张量。
    de_tokenizer: 目标语言的词汇表，用于将索引转换为单词。

    返回:
    list: 生成的目标语言句子列表。
    """
    model.eval()  # 将模型设置为评估模式

    src_mask = generate_src_mask(src, EN_VOCAB['<pad>'])  # 生成源序列的掩码

    # 使用<sos>标记初始化目标输入张量
    tgt_input = torch.full((src.size(0), 1), DE_VOCAB['<sos>'], dtype=torch.long, device=DEVICE)

    # 为批处理中的每个序列创建一个结束标志
    eos_flags = torch.zeros(src.size(0), dtype=torch.bool, device=DEVICE)

    # 对每个目标令牌进行推理
    with torch.no_grad():
        for _ in range(70):  # 最多生成70个令牌
            tgt_mask = generate_tgt_mask(tgt_input, DE_VOCAB['<pad>'])  # 生成目标序列的掩码
            output = model(src, tgt_input, src_mask, tgt_mask)
            next_tokens = output.argmax(2)[:, -1].unsqueeze(1)  # 选择概率最高的下一个令牌
            tgt_input = torch.cat((tgt_input, next_tokens), dim=1)  # 将下一个令牌添加到目标输入中

            # 更新已生成<eos>标记的序列的结束标志
            eos_flags |= (next_tokens.squeeze() == DE_VOCAB['<eos>'])
            # 如果所有序列都已生成<eos>或达到最大长度，则停止生成
            if torch.all(eos_flags):
                break

    # 将目标输入张量转换为翻译后的句子
    translated_sentences = []
    for i in range(tgt_input.size(0)):
        translated_tokens = []
        for token in tgt_input[i][1:]:  # 跳过第一个<sos>标记
            if token == DE_VOCAB['<eos>']:
                break  # 遇到<eos>标记时停止
            else:
                translated_tokens.append(DE_VOCAB.itos[token.item()])  # 将索引转换为单词

        translated_sentence = ' '.join(translated_tokens)  # 将单词列表连接成句子
        translated_sentences.append(translated_sentence)

    return translated_sentences


## 开启训练

In [37]:
best_val_loss = float('inf')  # 记录最佳验证损失值
train_losses = []  # 存储每个epoch的训练损失
val_losses = []  # 存储每个epoch的验证损失
bleu_scores = []  # 存储每个epoch的BLEU分数

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"Epoch {epoch}/{NUM_EPOCHS}")

    # 训练
    train_loss = train_epoch(model, train_dataloader, optimizer, criterion, DEVICE)
    train_losses.append(train_loss)

    # 验证
    val_loss, bleu_score = validate_epoch(model, val_dataloader, criterion, DEVICE)
    val_losses.append(val_loss)
    bleu_scores.append(bleu_score)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | BLEU Score: {bleu_score:.4f}")

    # 如果当前验证损失低于之前的最佳值，则保存模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')


Epoch 1/20


Train Loss: 5.8183 | Val Loss: 4.2099 | BLEU Score: 7.6717
Epoch 2/20


Train Loss: 3.8065 | Val Loss: 3.6510 | BLEU Score: 12.0534
Epoch 3/20


Train Loss: 3.3156 | Val Loss: 3.3265 | BLEU Score: 15.4623
Epoch 4/20


Train Loss: 2.9344 | Val Loss: 3.1600 | BLEU Score: 16.3336
Epoch 5/20


...

Train Loss: 1.7478 | Val Loss: 3.1507 | BLEU Score: 26.5843
Epoch 17/20


Train Loss: 1.7228 | Val Loss: 3.1590 | BLEU Score: 25.8857
Epoch 18/20


Train Loss: 1.7011 | Val Loss: 3.1746 | BLEU Score: 26.1551
Epoch 19/20


Train Loss: 1.6819 | Val Loss: 3.1813 | BLEU Score: 23.9430
Epoch 20/20


Train Loss: 1.6643 | Val Loss: 3.2126 | BLEU Score: 24.5360


## 评估test集

In [1]:
def evaluate_test_set_bleu(model, test_dataloader, de_tokenizer):
    """
    在测试集上评估模型的BLEU分数。

    参数:
    model: 训练好的Transformer模型。
    test_dataloader: 测试数据的数据加载器。
    de_tokenizer: 目标语言的词汇表。

    返回:
    float: 测试集的BLEU分数。
    """
    translated_sentences = []
    ground_truth_sentences = []

    for batch in tqdm(test_dataloader, desc="Evaluating"):
        src, tgt_output = batch
        src, tgt = src.to(DEVICE), tgt_output.to(DEVICE)
        tgt_sentences = [' '.join([DE_VOCAB.itos[token.item()] for token in sequence if token.item() not in [DE_VOCAB['<pad>'], DE_VOCAB['<sos>'], DE_VOCAB['<eos>']]]) for sequence in tgt_output]

        translations = inference(model, src, de_tokenizer)
        translated_sentences.extend(translations)
        ground_truth_sentences.extend([[tgt] for tgt in tgt_sentences])
        
    rand_index = random.randint(0, len(test_dataset))
    print("\n\nExample Sentence and its Translation")
    print("Source Sentence in English               :", ' '.join([EN_VOCAB.itos[i] for i in test_dataset[rand_index][0] if EN_VOCAB.itos[i] not in ['<pad>', '<sos>', '<eos>']]))
    print("Ground Truth Sentence in German          :", ground_truth_sentences[rand_index][0])
    print("Machine Translated Sentence in German    :", translated_sentences[rand_index])

    bleu_score = sacrebleu.corpus_bleu(translated_sentences, ground_truth_sentences)
    return bleu_score

# Usage example
test_bleu = evaluate_test_set_bleu(model, test_dataloader, de_tokenizer)
print("Test BLEU score:", test_bleu.score)


NameError: name 'model' is not defined

100%|██████████| 29000/29000 [00:00<00:00, 29507.32it/s]



Vocab Size English 10837

Vocab Size German 19214
Epoch 1/10


Train:   0%|          | 0/227 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (31) must match the size of tensor b (256) at non-singleton dimension 3